## Домашнее задание

Откуда берутся датасеты

Цель:

В этом ДЗ вы напишите свой парсер, который будет бегать по страничкам и автоматически что-то собирать.

Описание/Пошаговая инструкция выполнения домашнего задания:

По аналогии с занятием по парсингу данных, возьмите интересующий вас сайт, на котором можно пособирать какие-то данные (и при этом API не предоставляется).

Идеальный датасет должен иметь текстовое описание некоторого объекта и некоторую целевую переменную, соответствующую этому объекту. Например:

- Сайт новостей: текстовое описание — сама новость, целевая переменная — количество просмотров новости (можно поделить на число дней с момента даты публикации, чтобы получить «среднее число просмотров в день»);
- Сайт с товарами/книгами/фильмами: текстовое описание товара/книги/фильма + средний рейтинг в качестве целевой переменной;
- Блоги — тексты заметок + число просмотров;
- И любые другие ваши идеи, которые подходят под такой формат;

Напишите свой парсер, который будет бегать по страничкам и автоматически что-то собирать.
Не забывайте, что парсинг — это ответственное мероприятие, поэтому не бомбардируйте несчастные сайты слишком частыми запросами (можно ограничить число запросов в секунду при помощи time.sleep(0.3), вставленного в теле цикла).

При необходимости очистить датасет от мусора с помощью регулярных выражений.

Посчитать статистики по собранным данным и провести EDA собранных данных (в случае, если данные представляют собой текст — посчитать частотности слов, выявить наиболее частотные слова и т. п.).

In [3]:
import requests      
import numpy as np   
import pandas as pd  
import time          

import warnings
warnings.filterwarnings("ignore")

from fake_useragent import UserAgent
from bs4 import BeautifulSoup
from tqdm import tqdm

Пакет **[bs4 , a.k.a BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/)** (тут есть гиперссылка на лучшего друга человека — документацию) был назван в честь стишка про красивый суп из Алисы в стране чудес.

Красивый суп — это совершенно волшебная библиотека, которая из сырого и необработанного HTML кода страницы выдаст вам структурированный массив данных, по которому очень удобно искать необходимые теги, классы, атрибуты, тексты и прочие элементы веб страниц.

> Пакет под названием BeautifulSoup — скорее всего, не то, что нам нужно. Это третья версия (Beautiful Soup 3), а мы будем использовать четвертую. Нужно будет установить пакет beautifulsoup4. Чтобы было совсем весело, при импорте нужно указывать другое название пакета — bs4, а импортировать функцию под названием BeautifulSoup. В общем, сначала легко запутаться, но эти трудности нужно преодолеть.

С необработанным XML кодом страницы пакет также работает (XML — это исковерканый и превращённый в диалект, с помощью своих команд, HTML). Для того, чтобы пакет корректно работал с XML разметкой, придётся в довесок ко всему нашему арсеналу установить пакет xml.

## Парсинг

Собирать данные будем на новостном сайте Nice Matin - утренняя Ницца )

In [7]:
top_page = 'https://www.nicematin.com/'

In [8]:
UserAgent().chrome

'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36'

In [9]:
top_response = requests.get(top_page, headers={'User-Agent': UserAgent().chrome})
top_response

<Response [200]>

In [10]:
top_html = top_response.content

In [11]:
len(top_html)

293538

In [12]:
top_soup = BeautifulSoup(top_html, 'html.parser')

In [13]:
type(top_soup.prettify())

str

In [14]:
print(top_soup.prettify()[:1000])

<!DOCTYPE html>
<html class="nm" lang="fr">
 <head>
  <meta charset="utf-8"/>
  <link href="https://cdn.assets03.nicematin.com/build/favicons/freemium/nm_fav.d9370e02.png" rel="icon" type="image/png"/>
  <link href="https://cdn.assets03.nicematin.com/build/favicons/freemium/nm_fav.d9370e02.png" rel="shortcut icon" type="image/x-icon"/>
  <link href="https://cdn.assets03.nicematin.com/build/favicons/freemium/nm_fav.d9370e02.png" rel="apple-touch-icon"/>
  <link href="https://cdn.assets03.nicematin.com/build/favicons/freemium/nm_fav.d9370e02.png" rel="apple-touch-icon" sizes="57x57">
   <link href="https://cdn.assets03.nicematin.com/build/favicons/freemium/nm_fav.d9370e02.png" rel="apple-touch-icon" sizes="60x60">
    <link href="https://cdn.assets03.nicematin.com/build/favicons/freemium/nm_fav.d9370e02.png" rel="apple-touch-icon" sizes="72x72"/>
    <link href="https://cdn.assets03.nicematin.com/build/favicons/freemium/nm_fav.d9370e02.png" rel="apple-touch-icon" sizes="76x76"/>
    <lin

In [15]:
top_soup.html.head.title

<title>Actualités et infos en direct - Nice-Matin</title>

Напишем функцию для получения списка ссылок на странице

In [17]:
def get_links_list (page, tag, class_to_find):
    """
    Parameters:
    page: link to an html-page to find the links on;
    tag: <tag> to find;
    class_to_find: <class> to find; 
    
    Returns: list of links
    """
    response = requests.get(page, headers={'User-Agent': UserAgent().chrome})
    html = response.content
    soup = BeautifulSoup(html, 'html.parser')
    obj_list = soup.find_all(tag, attrs = {'class':class_to_find})
    links_list = [obj_item.attrs['href'] for obj_item in obj_list if obj_item.attrs['href'][0:25] == 'https://www.nicematin.com']
    return links_list

Соберем список тематических ссылок на первой странице Nice matin

In [19]:
theme_links_list = get_links_list(top_page, 'a', 'header-link-secondary')

In [20]:
theme_links_list

['https://www.nicematin.com/sujet/faits-de-societe',
 'https://www.nicematin.com/sujet/faits-divers',
 'https://www.nicematin.com/sujet/economie',
 'https://www.nicematin.com/sujet/politique',
 'https://www.nicematin.com/sujet/sport',
 'https://www.nicematin.com/sujet/sante',
 'https://www.nicematin.com/sujet/ogcnice',
 'https://www.nicematin.com/sujet/opinion',
 'https://www.nicematin.com/sujet/culture',
 'https://www.nicematin.com/sujet/jeux-olympiques',
 'https://www.nicematin.com/ville/nice',
 'https://www.nicematin.com/commune/saint-laurent-du-var',
 'https://www.nicematin.com/ville/cagnes',
 'https://www.nicematin.com/ville/cannes',
 'https://www.nicematin.com/ville/antibes',
 'https://www.nicematin.com/ville/grasse',
 'https://www.nicematin.com/ville/menton',
 'https://www.nicematin.com/ville/monaco',
 'https://www.nicematin.com/ville/vallees']

In [21]:
theme_links_list[0]

'https://www.nicematin.com/sujet/faits-de-societe'

Пройдем по всем тематическим ссылкам и соберем итоговый список ссылок на новостные статьи

In [23]:
allnews_links_list = []

In [24]:
for link in tqdm(theme_links_list):
    allnews_links_list = allnews_links_list + get_links_list(link, 'a', 'title-link')
    time.sleep(0.3)

100%|███████████████████████████████████████████| 19/19 [00:13<00:00,  1.36it/s]


In [25]:
len(allnews_links_list)

294

In [26]:
allnews_links_list[:10]

['https://www.nicematin.com/faits-de-societe/cinq-ans-apres-avoir-vecu-hors-du-temps-les-pompiers-reviennent-emus-a-notre-dame-962951',
 'https://www.nicematin.com/faits-de-societe/miss-martinique-elue-miss-france-2025-962857',
 'https://www.nicematin.com/faits-de-societe/miss-france-2025-pour-la-premiere-fois-une-candidate-de-plus-de-30-ans-962806',
 'https://www.nicematin.com/education/-le-but-est-de-savoir-ecrire-80-des-mots-francais-orthophoniste-a-antibes-camille-fenart-a-cree-des-jeux-pour-vaincre-les-troubles-de-l-orthographe-962794',
 'https://www.nicematin.com/faits-de-societe/decouvrez-le-grand-jt-des-territoires-du-samedi-14-decembre-962633',
 'https://www.nicematin.com/faits-de-societe/le-premier-qui-bouge-est-gay-associations-et-syndicats-alertent-sur-un-jeu-homophobe-962644',
 'https://www.nicematin.com/faits-divers/il-avait-porte-plusieurs-coups-de-couteau-a-sa-compagne-devant-leur-fille-de-3-ans-la-perpetuite-requise-contre-l-accuse-962608',
 'https://www.nicematin.com/

Посмотрим на первую ссылку в списке

In [28]:
allnews_links_list[0]

'https://www.nicematin.com/faits-de-societe/cinq-ans-apres-avoir-vecu-hors-du-temps-les-pompiers-reviennent-emus-a-notre-dame-962951'

попробуем получить детальную информацию по новости

In [30]:
response = requests.get(allnews_links_list[0], headers={'User-Agent': UserAgent().chrome})

In [31]:
response

<Response [200]>

In [32]:
html = response.content

In [33]:
soup = BeautifulSoup(html, 'html.parser')

In [34]:
soup.prettify()[:1000]

'<!DOCTYPE html>\n<html class="nm" lang="fr">\n <head>\n  <meta charset="utf-8"/>\n  <link href="https://cdn.assets03.nicematin.com/build/favicons/freemium/nm_fav.d9370e02.png" rel="icon" type="image/png"/>\n  <link href="https://cdn.assets03.nicematin.com/build/favicons/freemium/nm_fav.d9370e02.png" rel="shortcut icon" type="image/x-icon"/>\n  <link href="https://cdn.assets03.nicematin.com/build/favicons/freemium/nm_fav.d9370e02.png" rel="apple-touch-icon"/>\n  <link href="https://cdn.assets03.nicematin.com/build/favicons/freemium/nm_fav.d9370e02.png" rel="apple-touch-icon" sizes="57x57">\n   <link href="https://cdn.assets03.nicematin.com/build/favicons/freemium/nm_fav.d9370e02.png" rel="apple-touch-icon" sizes="60x60">\n    <link href="https://cdn.assets03.nicematin.com/build/favicons/freemium/nm_fav.d9370e02.png" rel="apple-touch-icon" sizes="72x72"/>\n    <link href="https://cdn.assets03.nicematin.com/build/favicons/freemium/nm_fav.d9370e02.png" rel="apple-touch-icon" sizes="76x76"

In [35]:
title = soup.find('h1', attrs = {'class':"article-title"}).text
title

'Cinq ans après avoir vécu "hors du temps", les pompiers reviennent émus à Notre-Dame'

In [36]:
excerpt = soup.find('h2', attrs = {'class':"article-excerpt"}).text.strip('\r\n ')
excerpt

'Ils l\'avaient quittée "totalement éventrée" et ne l\'avaient plus revue depuis cinq ans. Le chef de bataillon Peneaud et la capitaine Humbert, de la brigade de sapeurs-pompiers de Paris, sont restés "sans voix" dimanche en pénétrant de nouveau dans Notre-Dame.'

In [37]:
author = soup.find('span', attrs = {'class':"author"})
if author: author = author.text
author

'AFP'

In [38]:
published = soup.find('span', attrs = {'class':"published-at"})
if published: published = published.text
published

'Publié le 15/12/2024 à 17:07, mis à jour le 15/12/2024 à 17:07'

In [39]:
city = soup.find('a', attrs = {'class':"blue city"})
if city: city = city.text
city

'France'

In [40]:
full_text = soup.find('section', attrs = {'class':"article-block", 'id':'full-text'})
if full_text: full_text = full_text.text.strip('\ncommentaires ')
full_text

'"C\'est saisissant. Quand j\'ai passé la porte en rentrant, j\'ai eu ce même besoin de reprendre mon souffle tellement c\'est magnifique. Incroyable !": la capitaine Anne-Sixtine Humbert, réserviste pour la brigade de sapeurs-pompiers de Paris (BSPP) en tant qu\'experte patrimoine, a été emportée par la lumière inondant l\'édifice, sublimée par les rayons du soleil de ce dimanche hivernal.\nIl y a cinq ans, elle avait pénétré dans la cathédrale au lendemain de l\'incendie pour sauver quelque 40 ?uvres de Notre-Dame. "Tout était dans la pénombre, il n\'y avait plus rien, plus de lumière, à part ce trou béant", se souvient-elle, en se tenant sous ce plafond éventré, aux côtés du chef de bataillon David Peneaud, qui lui était intervenu dès le 15 avril, jour du terrible incendie.\n"Elle est plus belle qu\'avant, en fait. On a l\'impression qu\'il ne s\'est rien passé", glisse l\'officier, "happé par le plafond" dès son entrée dans la cathédrale.\nLes deux sapeurs-pompiers de Paris ont rev

Напишем функцию, возвращающую информацию из новости (title, author, published, region, excerpt, full_text) по url

In [42]:
def getNewsData (nlink):
    
    response = requests.get(nlink, headers={'User-Agent': UserAgent().chrome})
    html = response.content
    soup = BeautifulSoup(html, 'html.parser')

    title = soup.find('h1', attrs = {'class':"article-title"})
    if title: title = title.text

    author = soup.find('span', attrs = {'class':"author"})
    if author: author = author.text
    
    published = soup.find('span', attrs = {'class':"published-at"})
    if published: published = published.text

    city = soup.find('a', attrs = {'class':"blue city"})
    if city: city = city.text
    
    excerpt = soup.find('h2', attrs = {'class':"article-excerpt"})
    if excerpt: excerpt = excerpt.text.strip('\r\n ')

    full_text = soup.find('section', attrs = {'class':"article-block", 'id':'full-text'})
    if full_text: full_text = full_text.text.strip('\ncommentaires ')

    data_row = {'Title':title, 'Author':author, 'Published':published, 'Region':city,
                'Excerpt':excerpt, 'Full_text':full_text}   
    
    return data_row

In [43]:
allnews_links_list[0]

'https://www.nicematin.com/faits-de-societe/cinq-ans-apres-avoir-vecu-hors-du-temps-les-pompiers-reviennent-emus-a-notre-dame-962951'

In [44]:
data_row = getNewsData(allnews_links_list[0])

In [45]:
data_row

{'Title': 'Cinq ans après avoir vécu "hors du temps", les pompiers reviennent émus à Notre-Dame',
 'Author': 'AFP',
 'Published': 'Publié le 15/12/2024 à 17:07, mis à jour le 15/12/2024 à 17:07',
 'Region': 'France',
 'Excerpt': 'Ils l\'avaient quittée "totalement éventrée" et ne l\'avaient plus revue depuis cinq ans. Le chef de bataillon Peneaud et la capitaine Humbert, de la brigade de sapeurs-pompiers de Paris, sont restés "sans voix" dimanche en pénétrant de nouveau dans Notre-Dame.',
 'Full_text': '"C\'est saisissant. Quand j\'ai passé la porte en rentrant, j\'ai eu ce même besoin de reprendre mon souffle tellement c\'est magnifique. Incroyable !": la capitaine Anne-Sixtine Humbert, réserviste pour la brigade de sapeurs-pompiers de Paris (BSPP) en tant qu\'experte patrimoine, a été emportée par la lumière inondant l\'édifice, sublimée par les rayons du soleil de ce dimanche hivernal.\nIl y a cinq ans, elle avait pénétré dans la cathédrale au lendemain de l\'incendie pour sauver qu

Работает ! Осталось пройти в цикле по всем новостным ссылкам и сохранить полученные данные в pandas dataframe

In [47]:
df_news = pd.DataFrame(columns=['Title', 'Author', 'Published', 'Region','Excerpt', 'Full_text'])

In [48]:
for news_link in tqdm(allnews_links_list):
    data_row = getNewsData(news_link)
    df_news = pd.concat([df_news, pd.DataFrame([data_row])], ignore_index=True)
    time.sleep(0.3)

100%|█████████████████████████████████████████| 294/294 [04:15<00:00,  1.15it/s]


In [49]:
df_news.shape

(294, 6)

In [50]:
df_news.tail()

,Title,Author,Published,Region,Excerpt,Full_text
289,"""Sensibiliser le public à l’importance des for...",Romain Hugues - rhugues@nicematin.fr,"Publié le 09/12/2024 à 14:45, mis à jour le 09...",Vallées,"Depuis moins d’un an, Mendy et Marc-Olivier De...","Ils mangent, boivent, respirent et pensent bou..."
290,Municipale partielle: les urnes ont livré leur...,Aurélie Selvi,"Publié le 09/12/2024 à 11:15, mis à jour le 09...",Vallées,"Dans le village des Alpes-de-Haute-Provence, à...","Dans la cité médiévale d’Entrevaux, les cheval..."
291,Bientôt des panneaux solaires sur les toits du...,FAB. BONGIOVANNI,"Publié le 09/12/2024 à 10:00, mis à jour le 09...",Nice,Les élus du Bar-sur-Loup sont d’accord sur le ...,La dernière séance du conseil municipal du Bar...
292,Les urnes ont rendu leur verdict: trois candid...,Aurélie Selvi avec A. D.,"Publié le 08/12/2024 à 20:49, mis à jour le 08...",Nice,Renouveler trois sièges au conseil municipal a...,"Ce dimanche 8 décembre, la centaine de citoyen..."
293,"À quatre mois de ""Biot et les Templiers"", la c...",B. C.,"Publié le 08/12/2024 à 16:15, mis à jour le 08...",Nice,La manifestation aux 100.000 visiteurs se tien...,"Les 4, 5 et 6 avril, Biot retourne au XIIIe si..."


## EDA

Посмотрим на общие параметры собранного датасета

In [53]:
df_news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294 entries, 0 to 293
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Title      294 non-null    object
 1   Author     292 non-null    object
 2   Published  292 non-null    object
 3   Region     292 non-null    object
 4   Excerpt    292 non-null    object
 5   Full_text  294 non-null    object
dtypes: object(6)
memory usage: 13.9+ KB


Видим, что есть две записи с пропущенными значениями - посмотрим на них

In [55]:
df_news[df_news.isna().any(axis=1)]

,Title,Author,Published,Region,Excerpt,Full_text
203,Noël à Antibes Juan-les-Pins: les moments fort...,None,None,None,None,Le Village de Noël et les manèges pour enfants...
211,Noël à Antibes Juan-les-Pins: les moments fort...,None,None,None,None,Le Village de Noël et les manèges pour enfants...


Видим, что записи дублируются - удалим их из датасета

In [57]:
df_news.dropna(inplace = True)

In [58]:
df_news.info()

<class 'pandas.core.frame.DataFrame'>
Index: 292 entries, 0 to 293
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Title      292 non-null    object
 1   Author     292 non-null    object
 2   Published  292 non-null    object
 3   Region     292 non-null    object
 4   Excerpt    292 non-null    object
 5   Full_text  292 non-null    object
dtypes: object(6)
memory usage: 16.0+ KB


Числовых признаков в наборе данных нет, посмотрим на количество уникальных признаков

In [60]:
df_news.nunique()

Title        255
Author       113
Published    255
Region        18
Excerpt      251
Full_text    255
dtype: int64

В датасете собраны новости из 18 различных регионов, посмотрим на них

In [62]:
df_news['Region'].unique()

array(['France', 'Antibes', 'Draguignan', 'Cannes', 'Monde', 'Nice',
       'Région', 'Saint-Tropez', 'Monaco', 'Toulon', 'Grasse',
       'Fréjus/Saint-Raphaël', "Côte d'Azur", 'Var', 'Menton',
       'Brignoles', 'Vallées', 'Cagnes'], dtype=object)

Также у нас 112 авторов, посмотрим на ряд первых из списка

In [64]:
df_news['Author'].unique()[:21]

array(['AFP', 'La rédaction', 'Émilie Moulinemoulin@nicematin.fr',
       'rOMAIN HUGUES', 'La rédaction (avec AFP)', 'F.C.',
       'La rédaction avec AFP', 'Damien Allemand', 'S. N.', 'C. C.',
       'N. SA.', 'C.V.', 'Cedric Verany', 'M.G.', 'Christophe CIRONE',
       'Gaëlle Arama', 'AFP par Kenan AUGEARD', 'G. P', 'Karine Wenger',
       'Thibaut Parat', 'K.Wenger'], dtype=object)

## Итоги
Удалось собрать данные с новостного сайта Nice Matin, датасет получился небольшим и в нем не оказалось целевой переменной - но это Франция, здесь может не быть целевой переменной, но всегда есть багет, сыр и rosé...